In [1]:
from SimpleCV import *
from IPython.display import HTML
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob

toevoegingen

- matrix misschien veranderen naar/toevoegen: hoeveel pixels in stukje zwart
- blob.contours -- aantal hoeken
- naar 8x8 doen ipv 4x4
- edge detection


In [2]:
disp = Display(displaytype='notebook')
size = 32

imagelist = []
horizontalSplit=[]

# get all files and add them to tuple with the postal codes
for filename in glob.glob("../dataset-images/*.png"): 
    originalImage = Image(filename)
    image = originalImage.binarize().dilate().erode()
    invertedImage = image.invert().embiggen( (128,40), Color.WHITE, (3,3))
    answer = filename[18:22]
    imagelist.append((answer, image, invertedImage))

numbers = []
   
for image in imagelist:
    for i in range(4):
        sizeHeight = 40
        sizeWidth = 32
        digit = image[0][(0 if i+1 == 1 else i):i+1]
        normalImage = image[1].crop(i*size, 0, size, size*(i+1))
        invertedImage = image[2].crop(i*sizeWidth, 0, sizeWidth, sizeHeight*(i+1))
        numbers.append((digit, normalImage, invertedImage))

In [3]:
# # divide number images into pieces of 8x8 and create matrix. image is 32X32
# def createMatrix(image):
#     size = 4
#     m = np.zeros((8,8))

#     pieces = []
#     for y in range(8):
#         for x in range(8):
#             crop = image.crop(x*size, y*size, size, size)
#             pieces.append(crop)
#             m[x][y] = 1 if crop.meanColor()[0] > 80 else 0
#     return m.T

In [50]:
# divide number images into pieces of 8x8 and create matrix. image is 32X32
def createMatrix(image):
    size = 4
    m = np.zeros((8,8))

    for y in range(8):
        for x in range(8):
            crop = image.crop(x*size, y*size, size, size)
            pixels = crop.getNumpy()[:,:,0] / 255
            m[x][y] = pixels.sum()
    return m.T.astype(int)

In [53]:
image = numbers[0][1]
matrix = createMatrix(image)
matrix

array([[ 0,  0,  5, 13,  0,  0,  0,  0],
       [ 0,  0, 14,  6,  0,  0,  0,  0],
       [ 0,  4, 16,  4,  0,  0,  0,  0],
       [ 0,  7, 16,  1,  0,  0,  0,  0],
       [ 0,  8, 16, 16, 16, 13,  1,  0],
       [ 0,  5, 16,  7,  9, 16,  5,  0],
       [ 0,  1, 14, 12,  4, 16,  5,  0],
       [ 0,  0,  4, 16, 16,  9,  0,  0]])

In [54]:
# check if centroid pixel is black. If black return 1 else 0 
def isCentroidPixelBlack(image, centroid):
    img = image.crop(centroid[0], centroid[1], 1, 1)
    color = img.meanColor()
    return 0 if color[0] > 200 else 1

In [55]:
# get the blob from the image
def getBlob(image):
    image.show()
    blobs = image.findBlobs()
    image.draw(blobs, autocolor=True)
    return blobs[0]

def splitImage(image, centroidY, height):    
    croppedImageTop = image.crop(0,0,height,centroidY)
    blobTop = getBlob(croppedImageTop)
    centroidTop = blobTop.centroid()
    pixelBlackTop = isCentroidPixelBlack(croppedImageTop,centroidTop)
    
    croppedImageBottom = image.crop(0,centroidY, height,(height-centroidY))   
    blobBottom = getBlob(croppedImageBottom)
    centroidBottom = blobBottom.centroid()
    pixelBlackBottom = isCentroidPixelBlack(croppedImageBottom,centroidBottom)

    return (pixelBlackTop, pixelBlackBottom)


########### move to analysis ######## 
def createHeatMap(image):
    size = 8
    m = np.zeros((4,4))
    pieces = []
    for y in range(4):
        for x in range(4):
            crop = image.crop(x*size, y*size, size, size)
            pieces.append(crop)
            m[x][y] = crop.meanColor()[0]
    plt.imshow(m.T, cmap='Blues', interpolation='nearest')
    plt.show()

In [62]:
numbersDict = {
    "grid_1_1": [],
    "grid_1_2": [],
    "grid_1_3": [],
    "grid_1_4": [],
    "grid_1_5": [],
    "grid_1_6": [],
    "grid_1_7": [],
    "grid_1_8": [],
    "grid_2_1": [],
    "grid_2_2": [],
    "grid_2_3": [],
    "grid_2_4": [],    
    "grid_2_5": [],
    "grid_2_6": [],
    "grid_2_7": [],
    "grid_2_8": [],
    "grid_3_1": [],
    "grid_3_2": [],
    "grid_3_3": [],
    "grid_3_4": [],
    "grid_3_5": [],
    "grid_3_6": [],
    "grid_3_7": [],
    "grid_3_8": [],
    "grid_4_1": [],
    "grid_4_2": [],
    "grid_4_3": [],
    "grid_4_4": [],
    "grid_4_5": [],
    "grid_4_6": [],
    "grid_4_7": [],
    "grid_4_8": [],
    "grid_5_1": [],
    "grid_5_2": [],
    "grid_5_3": [],
    "grid_5_4": [],
    "grid_5_5": [],
    "grid_5_6": [],
    "grid_5_7": [],
    "grid_5_8": [],
    "grid_6_1": [],
    "grid_6_2": [],
    "grid_6_3": [],
    "grid_6_4": [],    
    "grid_6_5": [],
    "grid_6_6": [],
    "grid_6_7": [],
    "grid_6_8": [],
    "grid_7_1": [],
    "grid_7_2": [],
    "grid_7_3": [],
    "grid_7_4": [],
    "grid_7_5": [],
    "grid_7_6": [],
    "grid_7_7": [],
    "grid_7_8": [],
    "grid_8_1": [],
    "grid_8_2": [],
    "grid_8_3": [],
    "grid_8_4": [],
    "grid_8_5": [],
    "grid_8_6": [],
    "grid_8_7": [],
    "grid_8_8": [],
    "blob_area" : [],
    "blob_centroid_x" : [],
    "blob_centroid_y" : [],
    "blob_angle" : [],
    "blob_height": [],
    "blob_width": [],
#     "blob_amount_contours": [],
    "number_of_holes":[],
    "centroid_is_empty": [],
    "centroid_top_is_empty":[],
    "centroid_bottom_is_empty":[],
    "label" : []
}
    
for number in numbers:
    digit = number[0]
    image = number[1]
    invertedImage = number[2]
    invertedBlobCount = len(invertedImage.findBlobs()) -1
    blob = getBlob(image)
    matrix = createMatrix(image)
    centroid = blob.centroid()
    split = splitImage(image, centroid[1], blob.height())
#     image.save(disp)
    for y in range(8):
        for x in range(8):
            numbersDict["grid_" + str(y+1) + "_" + str(x +1)].append(matrix[y][x])

    
#     numbersDict["matrix_top_row"].append(sum(matrix[0]))
#     numbersDict["matrix_second_row"].append(sum(matrix[1]))
#     numbersDict["matrix_third_row"].append(sum(matrix[2]))
#     numbersDict["matrix_bottom_row"].append(sum(matrix[3]))
    numbersDict["blob_area"].append(blob.area())
    numbersDict["blob_centroid_x"].append(centroid[0])
    numbersDict["blob_centroid_y"].append(centroid[1])
    numbersDict["blob_angle"].append(blob.angle())
    numbersDict["blob_height"].append(blob.height())
    numbersDict["blob_width"].append(blob.width())
#     numbersDict["blob_amount_contours"].append(len(blob.contour()))
    numbersDict["number_of_holes"].append(invertedBlobCount)
    numbersDict["centroid_is_empty"].append(isCentroidPixelBlack(image, centroid))
    numbersDict["centroid_top_is_empty"].append(split[0])
    numbersDict["centroid_bottom_is_empty"].append(split[1])
    numbersDict["label"].append(number[0])    

df = pd.DataFrame(numbersDict)
df = df[[
#         "matrix_top_row", 
#          "matrix_second_row", 
#          "matrix_third_row", 
#          "matrix_bottom_row", 
        "grid_1_1",
        "grid_1_2",
        "grid_1_3",
        "grid_1_4",
        "grid_1_5",
        "grid_1_6",
        "grid_1_7",
        "grid_1_8",
        "grid_2_1",
        "grid_2_2",
        "grid_2_3",
        "grid_2_4",
        "grid_2_5",
        "grid_2_6",
        "grid_2_7",
        "grid_2_8",
        "grid_3_1",
        "grid_3_2",
        "grid_3_3",
        "grid_3_4",
        "grid_3_5",
        "grid_3_6",
        "grid_3_7",
        "grid_3_8",
        "grid_4_1",
        "grid_4_2",
        "grid_4_3",
        "grid_4_4",
        "grid_4_5",
        "grid_4_6",
        "grid_4_7",
        "grid_4_8",        
        "grid_5_1",
        "grid_5_2",
        "grid_5_3",
        "grid_5_4",
        "grid_5_5",
        "grid_5_6",
        "grid_5_7",
        "grid_5_8",
        "grid_6_1",
        "grid_6_2",
        "grid_6_3",
        "grid_6_4",
        "grid_6_5",
        "grid_6_6",
        "grid_6_7",
        "grid_6_8",
        "grid_7_1",
        "grid_7_2",
        "grid_7_3",
        "grid_7_4",
        "grid_7_5",
        "grid_7_6",
        "grid_7_7",
        "grid_7_8",
        "grid_8_1",
        "grid_8_2",
        "grid_8_3",
        "grid_8_4",
        "grid_8_5",
        "grid_8_6",
        "grid_8_7",
        "grid_8_8",
         "blob_area", 
         "blob_centroid_x", 
         "blob_centroid_y",
         "blob_angle",
         "blob_height",
         "blob_width",
#          "blob_amount_contours",
         "number_of_holes",
         "centroid_is_empty",
         "centroid_top_is_empty",
         "centroid_bottom_is_empty",
         "label"]]
df = df.sort_values("label")

In [63]:
# TEST : 4686: too many blobs

# testImage = Image("../dataset-images/4686.png")
# testImage = testImage.binarize().dilate().erode()
# blobs = testImage.findBlobs()
# testImage.draw(blobs, autocolor=True)
# testImage.save(disp)
# print blobs.area()

# testNums = []

# for i in range(4):
#     testNums.append(testImage.crop(i*size, 0, size, size*(i+1) ))

In [64]:
df.to_csv("../dataset-numpy/dataset_features.csv", sep=',', encoding='utf-8', index=False)

In [ ]:
#####move to feature analysis ####

df.describe()

In [11]:
# #list with tuples (digit, invertbloblength, counter) 
# ['0', 1, 4]
# ['0', 2, 173]
# ['1', 1, 198]
# ['2', 1, 193]
# ['2', 2, 2]
# ['3', 1, 194]
# ['3', 2, 5]
# ['4', 1, 171]
# ['4', 2, 14]
# ['5', 1, 184]
# ['5', 2, 3]
# ['6', 1, 60]
# ['6', 2, 135]
# ['7', 1, 196]
# ['7', 2, 3]
# ['8', 1, 27]
# ['8', 2, 62]
# ['8', 3, 91]
# ['9', 1, 60]
# ['9', 2, 144]

# ['0', 0, 4], 
# ['0', 1, 173], 
# ['1', 0, 198], 
# ['2', 0, 193],
# ['2', 1, 2]
# ['3', 0, 194],
# ['3', 1, 5],
# ['4', 1, 15], 
# ['4', 0, 171], 
# ['5', 0, 184], 
# ['5', 1, 3], 
# ['6', 1, 135], 
# ['6', 0, 60],
# ['7', 0, 196],
# ['7', 1, 3], 
# ['8', 0, 27],
# ['8', 1, 53],
# ['8', 2, 91], 
# ['9', 0, 60], 
# ['9', 1, 144], 
  


20

20

In [12]:
# #### naar feature analysis verplaatsen #######

# show labels with the holes and amount
df.groupby(['label', 'number_of_holes']).size()

label  number_of_holes
0      0                    4
       1                  173
1      0                  198
2      0                  193
       1                    2
3      0                  194
       1                    5
4      0                  171
       1                   15
5      0                  184
       1                    3
6      0                   60
       1                  135
7      0                  196
       1                    3
8      0                   27
       1                   62
       2                   91
9      0                   60
       1                  144
dtype: int64